In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from importlib import reload
import OptionPricers
reload(OptionPricers)
from OptionPricers import Black76Call, Black76Put, SABR
import pickle
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [11]:
# Data required from previous parts

# 1. Discount Curve (OIS & LIBOR)
with open('data/ois_discount_curve.pkl', 'rb') as f:
    ois_discount_curve = pickle.load(f)
with open('data/libor_discount_curve.pkl', 'rb') as f:
    libor_discount_curve = pickle.load(f)

# 2. Calibrated SABR params
sabr_alpha_df = pd.read_excel("data/model_calibration.xlsx", sheet_name="sabr_alpha", index_col=0)
sabr_alpha_df.columns = sabr_alpha_df.columns.astype('int64')
sabr_rho_df = pd.read_excel("data/model_calibration.xlsx", sheet_name="sabr_rho", index_col=0)
sabr_rho_df.columns = sabr_rho_df.columns.astype('int64')
sabr_nu_df = pd.read_excel("data/model_calibration.xlsx", sheet_name="sabr_nu", index_col=0)
sabr_nu_df.columns = sabr_nu_df.columns.astype('int64')

# 3. Swap Rate Curve
def swap_pv01(Tn, N, freq):
    Do = ois_discount_curve
    TN = Tn + N
    dc = 1.0/freq # day count fraction
    return dc * (Do(np.arange(TN,Tn,-dc)).sum())

def swap_curve(Tn, N, freq):
    Do = ois_discount_curve
    D = libor_discount_curve
    TN = Tn + N
    PV01 = swap_pv01(Tn, N, freq)
    dc = 1.0/freq # day count fraction
    PV_flt = sum([Do(i)*(D(i-dc)-D(i))/D(i) if i>dc \
                  else Do(i)*(1-D(i))/D(i) \
                for i in np.arange(TN,Tn,-dc)])
    return PV_flt / PV01

The PDF of swap rate can be derived from the 2nd order derivative of IRR-settled swaption price
\begin{equation*}
\begin{split}
    f(K) &= \frac{\partial^2 V^{IRR}}{\partial K^2}\dfrac{1}{D(0,T)\cdot IRR(K)}
\end{split}
\end{equation*}
Using static replication approach and let $h(K)=\dfrac{g(K)}{IRR(K)}$, we have
\begin{equation*}
\begin{split}
    V_0 &= D(0,T) \int_0^\infty g(K)f(K)dK \\
    &= \int_0^F h(K)\frac{\partial^2 V^{rec}_0}{\partial K^2}dK+\int_F^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK
\end{split}
\end{equation*}
For option (1) that pays $g(K) = K^{1/p}-0.04^{1/q}$
\begin{equation*}
\begin{split}
    V_0 &= D(0,T)g(F) + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
\end{split}
\end{equation*}
For option (2) that pays $g(K) = (K^{1/p}-0.04^{1/q})^+$, when $K <= x* = 0.04^{p/q},g(K) = 0$  

When $x* <= F$
\begin{equation*}
\begin{split}
    V_0 &= \int_{x*}^F h(K)\frac{\partial^2 V^{rec}_0}{\partial K^2}dK+\int_F^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK \\
    &= D(0,T)g(F) + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK \\
    &\;\;\;\;\;\;\;\;\;\;+ h'(x*)V^{rec}(x*)
\end{split}
\end{equation*}

When $x* > F$
\begin{equation*}
\begin{split}
    V_0 &= \int_{x*}^\infty h(K)\frac{\partial^2 V^{pay}_0}{\partial K^2}dK \\
    &= h'(x*)V^{pay}(x*) + \int_{x*}^\infty h''(K) V^{pay}(K) dK
\end{split}
\end{equation*}

Here,
\begin{equation*}
\begin{split}
    g(K) &= K^{1/p}-0.04^{1/q} \\
    g'(K) &= \dfrac{1}{p}K^{1/p-1}-0.04^{1/q} \\
    g''(K) &= \dfrac{(1-p)}{p^2}K^{1/p-2}-0.04^{1/q}
\end{split}
\end{equation*}

In [30]:
def IRR_0(K, m, N):
    return 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )

def IRR_1(K, m, N):
    return -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)

def IRR_2(K, m, N):
    return -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)

def g_0(K, p, q):
    # return K**(1.0/p)-0.04**(1.0/q)
    return K**0.25-0.04**0.5 # test

def g_1(K, p, q):
    # return 1.0/p*K**(1.0/p-1)-0.04**(1.0/q)
    return 0.25*K**(-0.75)-0.04**0.5 # test

def g_2(K, p, q):
    # return 1.0/p*(1.0/p-1)*K**(1.0/p-2)-0.04**(1.0/q)
    return (-0.75)*0.25*K**(-1.75)-0.04**0.5 # test

def h_0(K, m, N, p, q):
    return g_0(K, p, q) / IRR_0(K, m, N)

def h_1(K, m, N, p, q):
    return (IRR_0(K, m, N)*g_1(K, p, q) - g_0(K, p, q)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2

def h_2(K, m, N, p, q):
    return ((IRR_0(K, m, N)*g_2(K, p, q) - IRR_2(K, m, N)*g_0(K, p, q) - 2.0*IRR_1(K, m, N)*g_1(K, p, q))/IRR_0(K, m, N)**2 \
            + 2.0*IRR_1(K, m, N)**2*g_0(K, p, q)/IRR_0(K, m, N)**3)

def option_pricer_1(Tn, N, freq, p, q):
    F = swap_curve(Tn, N, freq)
    alpha = sabr_alpha_df[N][Tn]
    beta = 0.9
    rho = sabr_rho_df[N][Tn]
    nu = sabr_nu_df[N][Tn]
    F_max = F+0.02
    I_rec = quad(lambda x: h_2(x, freq, N, p, q) * Black76Put(F, 
                                                              x, 
                                                              SABR(F, x, Tn, alpha, beta, rho, nu), 
                                                              Tn),
                 0.0, F)[0]
    I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                               x, 
                                                               SABR(F, x, Tn, alpha, beta, rho, nu), 
                                                               Tn),
                 F, F_max)[0]
    return g_0(F, p, q) + IRR_0(F, freq, N)*(I_rec + I_pay)

def option_pricer_2(Tn, N, freq, p, q):
    F = swap_curve(Tn, N, freq)
    alpha = sabr_alpha_df[N][Tn]
    beta = 0.9
    rho = sabr_rho_df[N][Tn]
    nu = sabr_nu_df[N][Tn]
    F_max = F+0.02
    x_star = 0.04**(p/q) # g(K) > 0 when K > x_star
    if x_star <= F:
        I_rec = quad(lambda x: h_2(x, freq, N, p, q) * Black76Put(F, 
                                                                  x, 
                                                                  SABR(F, x, Tn, alpha, beta, rho, nu), 
                                                                  Tn),
                    x_star, F)[0]
        I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                                   x, 
                                                                   SABR(F, x, Tn, alpha, beta, rho, nu), 
                                                                   Tn),
                    F, F_max)[0]
        extra_term = h_1(x_star, freq, N, p, q) * Black76Put(F, 
                                                             x_star, 
                                                             SABR(F, x_star, Tn, alpha, beta, rho, nu), 
                                                             Tn)
        return g_0(F, p, q) + IRR_0(F, freq, N)*(extra_term + I_rec + I_pay)
    else:
        I_pay = quad(lambda x: h_2(x, freq, N, p, q) * Black76Call(F, 
                                                                   x, 
                                                                   SABR(F, x, Tn, alpha, beta, rho, nu), 
                                                                   Tn),
                    x_star, F_max)[0]
        extra_term = h_1(x_star, freq, N, p, q) * Black76Call(F, 
                                                              x_star, 
                                                              SABR(F, x_star, Tn, alpha, beta, rho, nu), 
                                                              Tn)
        return IRR_0(F, freq, N)*(extra_term + I_pay)


In [29]:
option_pricer_1(Tn=5, N=10, freq=2, p=4, q=2)

/var/folders/ys/nqq2g2_n2td1rcfs2z3h80zm0000gn/T/ipykernel_10165/3772583714.py:39: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  I_rec = quad(lambda x: h_2(x, freq, N, p, q) * Black76Put(F,


0.22583042065485365

In [31]:
option_pricer_2(Tn=5, N=10, freq=2, p=4, q=2)

0.24533197451904903

<!-- When payoff is
$$
    g(S) = (S^{1/p} - 0.04^{1/q})^+\;\;\;\;\text{ where } p=4,\;q=2
$$
Let $S^{1/p}=X, 0.04^{1/q}=K$, the payoff and option pricer turn into
\begin{equation*}
\begin{split}
    g(K, X) &= (X - K)^+ \\
    V_0 &= D(0,T) \int_0^\infty g(s)\cdot f(s)ds \\
    &= D(0,T) \int_K^\infty g(K, x)\cdot f(x^p)\cdot px^{p-1}dx
\end{split}
\end{equation*}

The probability density function of swap rate can be derived from the 2nd order derivative of IRR-settled swaption price:
\begin{equation*}
\begin{split}
    f(s) &= \dfrac{1}{D(0,T)\cdot IRR(s)}\frac{\partial^2 V^{IRR}(s)}{\partial s^2}
\end{split}
\end{equation*}

Then work out static replication formula
\begin{equation*}
\begin{split}
    V_0 &= D(0,T) \int_K^\infty g(K,x)\dfrac{1}{D(0,T)\cdot IRR(x^p)}\frac{\partial^2 V^{IRR}(x^p)}{\partial (x^p)^2}\cdot px^{p-1}dx
\end{split}
\end{equation*}
Let $h(x) = \dfrac{g(x)}{IRR(x^p)}, h(K) = 0$ as $g(K) = 0$, hence
\begin{equation*}
\begin{split}
    V_0 &= \int_K^\infty h(x)\frac{\partial^2 V^{pay}_0}{\partial x^2}dx \\
    &= -h(K)\frac{\partial V^{pay}_0(K)}{\partial K} + h'(K)V^{pay}_0(K) + \int_K^\infty h''(x)V^{pay}_0(x)dx \\
    &= h'(K)V^{pay}_0(K) + \int_K^\infty h''(x)V^{pay}_0(x)dx 
\end{split}
\end{equation*}
Here
\begin{equation*}
\begin{split}
    h(K) &= \dfrac{g(K)}{IRR(K^p)} \\
    h'(K) &= \dfrac{IRR(K^p)g'(K)-pK^{p-1}IRR'(K^p)g(K)}{IRR(K^p)^2} \\
    h''(K) &= \dfrac{IRR(K^p)g''(K)-p(p-1)K^{p-2}IRR'(K^p)g(K)-p^2K^{2p-2}IRR''(K^p)g(K)-2pK^{p-1}IRR'(K^p)g'(K)}{IRR(K^p)^2} \\
    &\;\;\;\;\;\;\;\;\;\; + \dfrac{2p^2K^{2p-2}IRR'(K^p)^2g(K)}{IRR(K^p)^3}
\end{split}
\end{equation*} -->

<!-- We first work out the probability density function using Leibniz's rule:
\begin{equation*}
\begin{split}
    I(k) &= \int_{u(k)}^{v(k)} g(k, s)ds \\
    \frac{\partial I(k)}{\partial x} &= g(k, v(k))\dfrac{dv}{dk} - g(k, u(k))\dfrac{du}{dk} + \int_{u(k)}^{v(k)} \frac{\partial g(k, s)}{\partial k}ds
\end{split}
\end{equation*}
Apply to $V^{pay}_0$ we get
\begin{equation*}
\begin{split}
    \frac{\partial V^{pay}_0}{\partial K} &= D(0,T)\{IRR(\infty^p)g(K, \infty)f(\infty)\dfrac{d\infty}{dK} - IRR(K^p)g(K,K)f(K)\dfrac{dK}{dK} - \int_K^\infty IRR(x^p)f(x)ds\} \\
    &= -D(0,T) \int_K^\infty IRR(x^p)f(x)ds \\
    \frac{\partial^2 V^{pay}_0}{\partial K^2} &= -D(0,T)\{IRR(\infty^p)f(\infty)\dfrac{d\infty}{dK} - IRR(K^p)f(K)\dfrac{dK}{dK} + \int_K^\infty \frac{\partial [IRR(x^p)f(x)]}{\partial K}ds\} \\
    &= D(0,T)\cdot IRR(K^p)\cdot f(K) \\
    => f(K) &= \dfrac{1}{D(0,T)\cdot IRR(K^p)}\frac{\partial^2 V^{pay}_0}{\partial K^2}
\end{split}
\end{equation*} -->
